# VacationPy

## Starter Code to Import Libraries and Load the Werater Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from datetime import datetime
import json
import os
import pprint
import plotly.express as px 
#import hvplot.pandas

# Import API key
#from api_keys import geoapify_key
#from api_keys import gkey

In [14]:
# Load the CSV file created in out 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df.dropna(inplace = True)

# Display sample data
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,George Town,MY,1667878666,5.4112,100.3354,87.73,74,20,5.75
1,1,Esim,GH,1667878578,4.8699,-2.2405,77.40,86,82,2.62
2,2,Bengkulu,ID,1667878786,-3.8004,102.2655,80.83,74,100,3.53
3,3,Glodeni,MD,1667878787,47.7708,27.5144,46.69,70,100,3.51
4,4,Kualakapuas,ID,1667878787,-3.0091,114.3876,86.07,66,98,4.81


### Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [21]:
fig = px.density_mapbox(city_data_df,(city_data_df["Lat"]), lon = (city_data_df["Lng"]), z= (city_data_df["Humidity"]), radius = 10,
                       center=dict(lat=0, lon=180), height = 600, zoom=1.5, mapbox_style="stamen-terrain")
fig.update_layout(title_text="Humidity Heatmap of Random Cities", margin={"r":50,"t":0,"l":50,"b":0})
fig.show()

#plt.savefig("output_data/Fig13_HumidityHeatMap.png")

<Figure size 432x288 with 0 Axes>

In [22]:
url = "https://api.geoapify.com/v2/places"

# Transform latitude and longitude to float
city_data_df["Lat"] = city_data_df["Lat"].astype(float)
city_data_df["Lng"] = city_data_df["Lng"].astype(float)


In [23]:
#Option for map with circles
fig = px.scatter_geo(city_data_df["City"], lat = (city_data_df["Lat"]), lon = (city_data_df["Lng"]), hover_name = "City", size = (city_data_df["Humidity"]), height = 900)
fig.update_layout(mapbox_style = "natural earth")
fig.update_layout(margin = {"r":0, "t":0, "l":0, "b":0})
fig.show()
#plt.savefig("output_data/Fig13.2_HumidityHeatMap.png")

<Figure size 432x288 with 0 Axes>

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condiditon.

In [27]:
my_weather_df = city_data_df.loc[(city_data_df['Max Temp'] >= 65) & (city_data_df['Max Temp'] <= 85) & (city_data_df['Humidity'] <= 40)]

my_weather_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Shingū,JP,1667878787,33.7333,135.9833,68.95,40,0,0.96
54,54,Mount Gambier,AU,1667878735,-37.8333,140.7667,79.66,40,0,1.05
83,83,Kolāras,IN,1667878812,25.2333,77.6000,78.64,40,61,3.76
109,109,New Norfolk,AU,1667878824,-42.7826,147.0587,80.91,27,49,3.00
112,112,Atar,MR,1667878825,20.5169,-13.0499,76.82,18,97,3.29


### Step 3: Create a new DataFrame called hotel_df

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,Shingū,JP,33.7333,135.9833,40,
54,Mount Gambier,AU,-37.8333,140.7667,40,
83,Kolāras,IN,25.2333,77.6000,40,
109,New Norfolk,AU,-42.7826,147.0587,27,
112,Atar,MR,20.5169,-13.0499,18,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates. 

In [29]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 10000,
    "categories": "accommodation.hotel",
    #"name": "hotel"
    #"types": "accomodation",
    #"keyword": "hotel",
    "apiKey":'f8649cf50b76470eaa75b191cf58bfc9'
}

#print start message
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lng},{lat}"
    params["filter"]= f'circle:{lng},{lat},10000'
    #params["bias"] = f'proximity:{lng},{lat}'

    base_url = "https://api.geoapify.com/v2/places?"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    #print(name_address.request.url)
       
    # convert to json
    name_address = name_address.json()
   
    #print(places_address)
    #print(json.dumps(name_address, indent=4, sort_keys=True))
   
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_name.append(np.nan)

    #print the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#print(json.dumps(places_address, indent=4, sort_keys=True))
hotel_df

Starting hotel search
Shingū - nearest hotel: 新宮セントラルポテンシャル
Mount Gambier - nearest hotel: Commodore on the Park
Kolāras - nearest hotel:  
New Norfolk - nearest hotel: Rosie's Inn
Atar - nearest hotel: Camping et Auberge Du Puigaudeau et Aziza
Yining - nearest hotel:  
Gharo - nearest hotel:  
Kongoussi - nearest hotel: Hotel La Sagesse Annexe
Abū Zabad - nearest hotel:  
Tessalit - nearest hotel:  
Arlit - nearest hotel:  
Blackwater - nearest hotel:  
Biltine - nearest hotel: فندق ثلاث نجوم بلتن
Kaduqli - nearest hotel: South Kordofan Hotel
Whyalla - nearest hotel: Whyalla Playford Apartments
Chandler - nearest hotel: Marriott Phoenix Chandler
Naukot - nearest hotel:  
Rīngas - nearest hotel:  
Port Hedland - nearest hotel: Hospitality Port Hedland
Kontagora - nearest hotel:  
Abéché - nearest hotel: فندق قيصر
Mirpur Mathelo - nearest hotel:  
Oyama - nearest hotel: 小山第一国際ホテル


,City,Country,Lat,Lng,Humidity,Hotel Name
6,Shingū,JP,33.7333,135.9833,40,新宮セントラルポテンシャル
54,Mount Gambier,AU,-37.8333,140.7667,40,Commodore on the Park
83,Kolāras,IN,25.2333,77.6000,40,
109,New Norfolk,AU,-42.7826,147.0587,27,Rosie's Inn
112,Atar,MR,20.5169,-13.0499,18,Camping et Auberge Du Puigaudeau et Aziza
156,Yining,CN,29.0361,114.5636,35,
157,Gharo,PK,24.7424,67.5856,39,
169,Kongoussi,BF,13.3258,-1.5347,15,Hotel La Sagesse Annexe
186,Abū Zabad,SD,12.3500,29.2500,22,
230,Tessalit,ML,20.1986,1.0114,19,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

Only some of the cities listed have hotels, so not all of the pins have a location. 

In [36]:
fig = px.scatter_mapbox(hotel_df, (hotel_df["Lat"]), lon = (hotel_df["Lng"]), hover_name=(hotel_df["Hotel Name"]), hover_data=["City", "Country"],
                       center=dict(lat=0, lon=180), height = 600, zoom=1.5, mapbox_style="stamen-terrain")
fig.update_layout(title_text="Hotel's for Ideal Cities", margin={"r":50,"t":0,"l":50,"b":0})

fig.show()
#plt.savefig("output_data/Fig14_Hotelsforidealcities.png")

<Figure size 432x288 with 0 Axes>